In [3]:
import sys
sys.version

'3.5.4 |Anaconda, Inc.| (default, Nov  9 2017, 00:39:12) \n[GCC 7.2.0]'

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
#import cv2

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.layers import Dropout,GaussianDropout
from keras.layers import GaussianNoise
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.utils import to_categorical
from keras.models import load_model
from keras.constraints import max_norm
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
#from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import StandardScaler
%matplotlib inline

Using TensorFlow backend.


In [5]:
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten

In [6]:
import tensorflow,keras
print(tensorflow.__version__, keras.__version__)

1.3.0 2.0.5


In [7]:
import types
import pandas as pd
from ibm_botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_439f896e860a45e8ab03c3254fac296b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='o4GnO1ev42cUPCz_5NFqcQ4SZySpmfFNpc_7NtE0OKGz',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_439f896e860a45e8ab03c3254fac296b.get_object(Bucket='courseraproject-donotdelete-pr-b3bunbszncvhtk',Key='sign_mnist_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()



,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [8]:
body = client_439f896e860a45e8ab03c3254fac296b.get_object(Bucket='courseraproject-donotdelete-pr-b3bunbszncvhtk',Key='sign_mnist_test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

d_test = pd.read_csv(body)
d_test.head()



,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,...,138,148,127,89,82,96,106,112,120,107
1,5,126,128,131,132,133,134,135,135,136,...,47,104,194,183,186,184,184,184,182,180
2,10,85,88,92,96,105,123,135,143,147,...,68,166,242,227,230,227,226,225,224,222
3,0,203,205,207,206,207,209,210,209,210,...,154,248,247,248,253,236,230,240,253,255
4,3,188,191,193,195,199,201,202,203,203,...,26,40,64,48,29,46,49,46,46,53


In [ ]:
df=pd.read_csv('/Users/anubhavjain/Desktop/sign-language-mnist/sign_mnist_train.csv')
d_test=pd.read_csv('/Users/anubhavjain/Desktop/sign-language-mnist/sign_mnist_test.csv')

In [9]:
df.loc[0:2,:]

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195


In [10]:
t=np.array(df.loc[0,:])

In [11]:
x_train=np.array(df[df.columns[df.columns!='label']])
x_label=np.array(df['label'])

In [12]:
y_test=np.array(d_test[d_test.columns[d_test.columns!='label']])
y_label=np.array(d_test['label'])

In [13]:
x_label=to_categorical(x_label)
y_label=to_categorical(y_label)

In [14]:
#x_train.shape
#np.unique(y_label)
no_classes=y_label.shape[1]

In [15]:
#x_train.shape
y_label.shape

(7172, 25)

In [16]:
datagen = ImageDataGenerator(rotation_range=20,width_shift_range=0.3,height_shift_range=0.15)

In [17]:
x_train=x_train.reshape(x_train.shape[0],28,28,1)
y_test=y_test.reshape(y_test.shape[0],28,28,1)

In [18]:
x_train = x_train.astype('float32')
y_test = y_test.astype('float32')

In [19]:
x_train=x_train/255
y_test=y_test/255

In [ ]:
print(x_train[0])

In [20]:
datagen.fit(x_train)

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/keras/preprocessing/image.py:648: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (27455, 28, 28, 1) (1 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


In [ ]:
#scalar = StandardScaler()

In [ ]:
#scalar.fit(x_train)

In [ ]:
#x_train = scalar.transform(x_train)
#y_test = scalar.transform(y_test)

In [ ]:
#x_train=x_train.reshape(x_train.shape[0],28,28,1)
#y_test=y_test.reshape(y_test.shape[0],28,28,1)

In [21]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': 'o4GnO1ev42cUPCz_5NFqcQ4SZySpmfFNpc_7NtE0OKGz',
    'IAM_SERVICE_ID': 'iam-ServiceId-446d7179-2f7f-4806-9a88-16e8d4a939fc',
    'ENDPOINT': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'courseraproject-donotdelete-pr-b3bunbszncvhtk',
    'FILE': 'gesture_to_text.h5'
}

In [29]:
model=Sequential()

In [ ]:
model=load_model('gesture_to_text.h5')

In [30]:
model.add(GaussianNoise(0.2,input_shape=(28,28,1)))
model.add(Conv2D(64,(5,5),strides=(1,1),activation='linear',input_shape=(28,28,1)))
#model.add(Activation('relu'))###
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.2))###
model.add(MaxPooling2D((2,2),strides=(2,2)))

model.add(Conv2D(128,(2,2),strides=(1,1),activation='linear'))
#model.add(Activation('relu'))###
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.2))###
model.add(MaxPooling2D((2,2),strides=(2,2)))

model.add(Conv2D(256,(2,2),strides=(1,1),activation='linear'))
#model.add(Activation('relu'))###
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.2))###
model.add(MaxPooling2D((2,2),strides=(1,1)))

model.add(Conv2D(512,(2,2),strides=(1,1),activation='linear'))
#model.add(Activation('relu'))###
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.2))###
model.add(MaxPooling2D((2,2),strides=(2,2)))

model.add(Flatten())

#,kernel_initializer='normal',kernel_constraint=max_norm(3))
#model.add(Dropout(0.1))
model.add(Dense(300,activation='linear'))
model.add(BatchNormalization())
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
#model.add(GaussianDropout(0.2))
model.add(Dense(300,activation='linear'))
model.add(BatchNormalization())
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(Dense(300,activation='linear'))
#model.add(BatchNormalization())
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(Dense(300,activation='linear'))
#model.add(BatchNormalization())
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
#model.add(GaussianDropout(0.2))
model.add(Dense(300,activation='linear'))
#model.add(BatchNormalization())
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(Dense(300,activation='linear'))
model.add(BatchNormalization())
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
#model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(no_classes,activation='softmax'))

In [31]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,x_label,validation_data=(y_test,y_label),epochs=20,batch_size=19,shuffle=True,callbacks=[TensorBoard(log_dir='/tmp/gesture_to_text')])

In [116]:
model.fit_generator(datagen.flow(x_train, x_label, batch_size=19),steps_per_epoch=len(x_train)/19,validation_data=(y_test,y_label),epochs=2000,callbacks=[TensorBoard(log_dir='/tmp/gesture_to_text3')])

Epoch 1/2
1445/1445 [==============================] - 45s - loss: 0.1134 - acc: 0.9620 - val_loss: 0.0052 - val_acc: 0.9990
Epoch 2/2
1445/1445 [==============================] - 45s - loss: 0.1149 - acc: 0.9623 - val_loss: 0.0043 - val_acc: 0.9990


In [124]:
model.evaluate(y_test,y_label)

7040/7172 [============================>.] - ETA: 0s

[0.0043019312369147042, 0.99902398215281651]

In [ ]:
#img = cv2.imread('/Users/anubhavjain/Desktop/sign-language-mnist/outpy1.png',0)

In [150]:

body = client_439f896e860a45e8ab03c3254fac296b.get_object(Bucket='courseraproject-donotdelete-pr-b3bunbszncvhtk',Key='outpy3.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_13 = pd.read_csv(body)
df_data_13.head()



,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,18,19,20,21,22,23,24,25,26,27
0,0,0.564706,0.596078,0.607843,0.619608,0.619608,0.639216,0.639216,0.666667,0.666667,...,0.741176,0.745098,0.749020,0.760784,0.772549,0.788235,0.792157,0.796078,0.800000,0.760784
1,1,0.596078,0.631373,0.635294,0.658824,0.666667,0.674510,0.682353,0.698039,0.694118,...,0.788235,0.792157,0.792157,0.803922,0.819608,0.835294,0.792157,0.800000,0.862745,0.878431
2,2,0.588235,0.627451,0.635294,0.639216,0.662745,0.666667,0.682353,0.694118,0.694118,...,0.792157,0.788235,0.788235,0.800000,0.819608,0.827451,0.839216,0.854902,0.862745,0.878431
3,3,0.584314,0.627451,0.631373,0.650980,0.654902,0.666667,0.670588,0.701961,0.698039,...,0.788235,0.780392,0.788235,0.803922,0.807843,0.827451,0.831373,0.854902,0.862745,0.866667
4,4,0.576471,0.619608,0.639216,0.650980,0.647059,0.666667,0.686275,0.701961,0.698039,...,0.784314,0.780392,0.780392,0.807843,0.815686,0.827451,0.843137,0.847059,0.862745,0.874510


In [151]:
pic_array = np.empty((28, 28))
for i in range(28):
    for j in range(1,29):
        pic_array[i][j-1] = df_data_13.loc[i][j]
pic_array,pic_array.shape

(array([[ 0.56470588,  0.59607843,  0.60784314,  0.61960784,  0.61960784,
          0.63921569,  0.63921569,  0.66666667,  0.66666667,  0.67843137,
          0.68627451,  0.69803922,  0.70196078,  0.71372549,  0.72156863,
          0.7372549 ,  0.7372549 ,  0.7372549 ,  0.74117647,  0.74509804,
          0.74901961,  0.76078431,  0.77254902,  0.78823529,  0.79215686,
          0.79607843,  0.8       ,  0.76078431],
        [ 0.59607843,  0.63137255,  0.63529412,  0.65882353,  0.66666667,
          0.6745098 ,  0.68235294,  0.69803922,  0.69411765,  0.71764706,
          0.71764706,  0.73333333,  0.74117647,  0.74509804,  0.76078431,
          0.76862745,  0.77254902,  0.77647059,  0.78823529,  0.79215686,
          0.79215686,  0.80392157,  0.81960784,  0.83529412,  0.79215686,
          0.8       ,  0.8627451 ,  0.87843137],
        [ 0.58823529,  0.62745098,  0.63529412,  0.63921569,  0.6627451 ,
          0.66666667,  0.68235294,  0.69411765,  0.69411765,  0.72156863,
          0.73

In [ ]:
#pic_array = scalar.transform(pic_array)

In [152]:
pic_array = pic_array.reshape(1,28,28,1)
pic_array.shape
#pic_array[0]

(1, 28, 28, 1)

In [153]:
t = model.predict(pic_array)
y = np.nonzero(t == max(t[0]))
t[0]

array([  1.71651837e-09,   4.71726693e-08,   4.39606287e-04,
         1.58680558e-09,   1.26753008e-08,   1.63506456e-06,
         5.92637625e-05,   9.97822642e-01,   2.27873382e-08,
         1.79007437e-14,   1.11838122e-10,   9.89485756e-08,
         1.07898501e-09,   3.87909843e-11,   5.87951945e-06,
         3.22529668e-05,   1.58821081e-03,   6.00714825e-11,
         5.53618748e-11,   3.70835587e-05,   5.68543934e-10,
         6.37273220e-11,   2.09206008e-09,   1.32571085e-05,
         4.15557970e-08], dtype=float32)

In [154]:
y[1][0]

7

In [155]:
chr(y[1][0] + 65)

'H'

In [ ]:
t_before=time.time()
scores=model.evaluate(y_test,y_label,verbose=0)
t_after=time.time()

In [ ]:
scores[1]

In [ ]:
(t_after-t_before)

In [57]:
model.save('gesture_to_text.h5')